In [ ]:
"""
\********************************************************************************
* Copyright (c) 2024 the Qrisp authors
*
* This program and the accompanying materials are made available under the
* terms of the Eclipse Public License 2.0 which is available at
* http://www.eclipse.org/legal/epl-2.0.
*
* This Source Code may also be made available under the following Secondary
* Licenses when the conditions for such availability set forth in the Eclipse
* Public License, v. 2.0 are satisfied: GNU General Public License, version 2
* with the GNU Classpath Exception which is
* available at https://www.gnu.org/software/classpath/license.html.
*
* SPDX-License-Identifier: EPL-2.0 OR GPL-2.0 WITH Classpath-exception-2.0
********************************************************************************/
"""

import numpy as np 
import scipy
import matplotlib.pyplot as plt 
from qrisp import *
from qrisp.qaoa import QAOABenchmark
from encode_PBS_problem import cost_coeff #, PBS_graph, N, M, 
from classical_cost_func import format_coeffs, cost_function

N = 4
Phi = [(1,0),(2,0),(3,1)]

PBS_graph = nx.DiGraph()
PBS_graph.add_edges_from(Phi)
M = PBS_graph.number_of_nodes()

tot_coeff = format_coeffs(cost_coeff, N)
cl_cost = cost_function(tot_coeff, PBS_graph)

benchmark_data = QAOABenchmark.load("data/constrainedQAOA_(1,0),(2,0),(3,1)_4")
#benchmark_data = QAOABenchmark.load("data/constrainedQAOA_(1,0),(2,0),(3,0),(4,1),(5,1),(6,1)_4")
#benchmark_data = QAOABenchmark.load("data/constrainedQAOA_(1,0),(2,0),(3,0),(4,1)_5")
#benchmark_data = QAOABenchmark.load("data/QUBO_QAOA_(1,0),(2,0),(3,1)_4")

optimal_solution = benchmark_data.optimal_solution
layers = benchmark_data.layer_depth
depth = benchmark_data.circuit_depth
results = benchmark_data.counts

In [ ]:
def ProbabilityApproximation10(res,cl_cost,optimal_solution):
    prob = 0
    optiomal_cost = cl_cost({optimal_solution:1})
    for k,v in res.items():
        c = cl_cost({k:1})
        if c < 1.1*optiomal_cost:
            prob += v
    return prob

def Approximation(res,cl_cost,optimal_solution):
    return cl_cost(res)/cl_cost(optimal_solution)

def ProbabilityArrgegated(res,cl_cost):
    costs = list(set([round(cl_cost({item[0]:1.0}),2) for item in res.items()]))
    probs = [0]*len(costs)
    for k,v in res.items():
        cost = round(cl_cost({k:1.0}),2)
        probs[costs.index(cost)] += v
    return costs, probs

x1, y1 = ProbabilityArrgegated(results[3],cl_cost)

x2 = layers
y2 = [ProbabilityApproximation10(res,cl_cost,optimal_solution) for res in results]
#y2 = depth


In [ ]:

fig, (ax1, ax2) = plt.subplots(1,2, figsize = (12.5, 4))

# First Plot (ProbabilityArrgegated)
ax1.set_xlabel("Cost", fontsize=15, color="#444444", fontname="Segoe UI")
ax1.set_ylabel("Probability", fontsize=15, color="#444444", fontname="Segoe UI")
ax1.set_xticks(ticks = range(8, 27, 4))
ax1.bar(x1, y1, width=0.1, color='#20306f')
#ax1.legend(fontsize=14, labelcolor='linecolor')
ax1.grid()


# Second Plot (ProbabilityApproximation10)
ax2.set_xlabel("Layers", fontsize=15, color="#444444", fontname="Segoe UI")
ax2.set_ylabel("Probability", fontsize=15, color="#444444", fontname="Segoe UI")
ax2.set_xticks(ticks = range(0, 5, 1))
ax2.plot(x2, y2, c='#20306f', marker="o", linestyle='solid', linewidth=3, zorder=3)
#ax2.legend(fontsize=14, labelcolor='linecolor')
ax2.grid()

# Show both plots side by side
plt.tight_layout()

#plt.savefig("data/constrainedQAOA_(1,0),(2,0),(3,1)_4_plot.pdf", bbox_inches = "tight")

plt.show()